 Mean Field VI for GMM assuming all model parameters are random

In [ ]:
%matplotlib inline
# %matplotlib notebook
%config Completer.use_jedi = False
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from scipy.special import logsumexp
from scipy.special import softmax
from scipy.special import betaln
from scipy.special import beta

from scipy.stats import multivariate_normal 
from scipy.stats import invwishart 

from sklearn import metrics
from scipy.special import psi


We want to compute the intractable GMM posterior:
$$
    p(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi}|\mathbf{X}) = \frac{p(\mathbf{X},\boldsymbol{\mu},\boldsymbol{\Sigma},\mathbf{z},\boldsymbol{\pi})}{p(\mathbf{X})}
$$

Introduce approximating distribution $q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\pi})$ where we use KL divergence as a cost function 
$$
    \mathbb{KL}(q||p) = \sum_{\mathbf{z},\boldsymbol{\mu}}q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi})\log\frac{q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi})}{p(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi}|\mathbf{X})}
$$

Objective : 
$$\begin{align}
    \mathcal{J}(q) &= \mathbb{KL}(q||p) = \int\int\int\sum_{\mathbf{z}}q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\pi})\log\frac{q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi})}{p(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi}|\mathbf{X})}d\boldsymbol{\mu}d\boldsymbol{\pi} d\boldsymbol{\Sigma}\\\\

    &=  \int\int\int\sum_{\mathbf{z}}q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi})\log q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi})d\boldsymbol{\mu}d\boldsymbol{\Sigma}d\boldsymbol{\pi} - \int\int\int\sum_{\mathbf{z}}q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi})\log p(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi}|\mathbf{X})d\boldsymbol{\mu}d\boldsymbol{\pi}d\boldsymbol{\Sigma}\\\\

    &= \Big\langle \log q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi}) \Big\rangle_{q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi})} - \Big\langle \log p(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi}|\mathbf{X}) \Big\rangle_{q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi})}\\

    &= \Big\langle \log q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi}) \Big\rangle_{q(\mathbf{z}, 
       \boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi})} -
       \Big\langle \log p(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi},\mathbf{X}) \Big\rangle_{q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi})} + 
       \Big\langle  \log p(\mathbf{X}) \Big\rangle_{q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi})}\\\\

    &= \Big\langle  \log \frac{q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi})}{p(\mathbf{X})} \Big\rangle_{q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi})}  + 
       \log p(\mathbf{X}) \\\\
    
    &= \mathbb{KL}\Big(q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi}) || p(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi},\mathbf{X})\Big) + \underbrace{\log p(\mathbf{X})}_{\text{constant}}
\end{align}$$

Objective can simplify to
$$\begin{align}
    q* &\leftarrow \argmin_{q} \left[  \mathbb{KL}\Big(q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi}) || p(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi},\mathbf{X})\Big)  + \underbrace{\log p(\mathbf{X})}_{\text{constant}} \right]\\
        &= \argmin_{q} \left[ \mathbb{KL}\Big(q(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi}) || p(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi},\mathbf{X})\Big) \right]
\end{align}$$

---

## Mean Field 

Mean Field (Structured) Independence Simplifying Assumption
$$
    q(\boldsymbol{\pi},\boldsymbol{\mu},\boldsymbol{\Sigma},\mathbf{z}) = 
        \underbrace{q(\boldsymbol{\pi})}_{\text{Dirichlet}}
        \underbrace{q(\boldsymbol{\mu},\boldsymbol{\Sigma})}_{\text{NIW}}
        \underbrace{q(\mathbf{z})}_{\text{Cat}}
$$

Update Steps 

$$\begin{align}
    &\log q(\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi}) = \Big\langle \log p(\mathbf{z},\mu,\Sigma,\mathbf{X}) \Big\rangle_{q(\mathbf{z})}  + \text{const}\\\\
    
    &\log q(\mathbf{z}) = \Big\langle \log p(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\mathbf{X}) \Big\rangle_{q(\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi})} + \text{const}
\end{align}$$

Posterior
$$
    p(\mathbf{X},\boldsymbol{\mu},\mathbf{z},\boldsymbol{\pi},\boldsymbol{\Sigma}) \propto p(\mathbf{X}|\boldsymbol{\mu},\boldsymbol{\Sigma},\mathbf{z})p(\boldsymbol{\mu}|\boldsymbol{\Sigma})p(\boldsymbol{\Sigma})p(\mathbf{z}|\boldsymbol{\pi})p(\boldsymbol{\pi})
$$

### Updates for variational posterior of model parameters $q(\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi})$

$$\begin{align}
    \log q(\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi}) &= \Big\langle \log p(\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma},\mathbf{X}) \Big\rangle_{q(\mathbf{z})}  + \text{const}\\\\

    &= \Big\langle \log p(\mathbf{X}|\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma}) \Big\rangle_{q(\mathbf{z})} 
       + \Big\langle \log p(\boldsymbol{\mu}|\boldsymbol{\Sigma})p(\boldsymbol{\Sigma})\Big\rangle_{q(\mathbf{z})} 
       + \Big\langle \log p(\mathbf{z}|\boldsymbol{\pi})p(\boldsymbol{\pi}) \Big\rangle_{q(\mathbf{z})} 
       + \text{const}\\\\

    &= \underbrace{\log p(\boldsymbol{\Sigma}) + \log p(\boldsymbol{\mu}|\boldsymbol{\Sigma}) 
       + \Big\langle \log p(\mathbf{X}|\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma})}_{\text{Terms containing $\mu,\Sigma$}} \Big\rangle_{q(\mathbf{z})}
       + \underbrace{\Big\langle \log p(\mathbf{z}|\boldsymbol{\pi}) \Big\rangle_{q(\mathbf{z})} + \log p(\boldsymbol{\pi})}_{\text{Terms containing $\pi$}}
\end{align}$$

Can separate RHS into terms containing $\pi$ and $\mu$

#### Update Step for $q(\boldsymbol{\pi})$

$$\begin{align}
    \log q(\boldsymbol{\pi}) &= \log p(\boldsymbol{\pi}) + \Big\langle \log p(\mathbf{z}|\boldsymbol{\pi}) \Big\rangle_{q(\mathbf{z})} + \text{const} \\

    &= \log \text{Dir}(\boldsymbol{\pi}|\boldsymbol{\alpha}_0) + \Big\langle \sum_n^N \log p(z_n|\boldsymbol{\pi}) \Big\rangle_{q(\mathbf{z})} + \text{const} \\

    &= \log \Big[ \frac{1}{B(\alpha_0)}\prod_k^K \pi_k^{\alpha_0 - 1} \Big] + \Big\langle \sum_n^N \sum_k^K \log \pi_{k}^{z_{nk}} \Big\rangle_{q(\mathbf{z})} + \text{const} \\
       
    &= \underbrace{-\log B(\alpha_0)}_{\text{Independent of $\pi$}} + \sum_k^K \log \pi_k^{\alpha_0 - 1} +
       \Big\langle \sum_k^K \sum_n^N z_{nk}\log \pi_{k} \Big\rangle_{q(\mathbf{z})} + \text{const} \\

    &=   \sum_k^K (\alpha_0 - 1) \log \pi_k + \sum_k^K \sum_n^N \Big\langle z_{nk} \Big\rangle_{q(\mathbf{z})} \log \pi_{k} + \text{const}, \quad -\log B(\alpha_0) \text{ absorbed into const}\\

    &= \sum_k^K (\alpha_0 - 1) \log \pi_k + \sum_k^K \sum_n^N r_{nk} \log \pi_{k} + \text{const}\\

    &= \sum_k^K \Big[ (\alpha_0 - 1) \log \pi_k + \log \pi_{k} \sum_n^N r_{nk} \Big] + \text{const}\\

    &= \sum_k^K \Big[ (\alpha_0 + N_k -1) \log \pi_k  \Big] + \text{const}, \quad \text{where } N_k = \sum_n^N r_{nk} \\

    \implies & \boxed{q(\boldsymbol{\pi}) = \prod_k^K \pi_k^{\alpha_0 + N_k - 1} + \text{const} = \text{Dir}(\boldsymbol{\pi}|\boldsymbol{\alpha}_0 + N), \quad \text{where } N = [N_0,...,N_K]}\\

\end{align}$$

#### Update step for $q(\boldsymbol{\mu},\boldsymbol{\Sigma})$

$$\begin{align}
    \log q(\boldsymbol{\mu},\boldsymbol{\Sigma}) &= \log p(\boldsymbol{\Sigma}) + \log p(\boldsymbol{\mu}|\boldsymbol{\Sigma}) + \Big\langle \log p(\mathbf{X}|\mathbf{z},\boldsymbol{\mu},\boldsymbol{\Sigma})\Big\rangle_{q(\mathbf{z})} + \text{const}\\

    &= \sum_k^K \log p(\boldsymbol{\Sigma}) + \sum_k^K \log p(\boldsymbol{\mu}|\boldsymbol{\Sigma}) + \Big\langle \sum_n^N \sum_k^K \log p(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)^{z_{nk}}\Big\rangle_{q(\mathbf{z})} + \text{const}\\

    &= \sum_k^K \Big[ 
            \log p(\boldsymbol{\Sigma}_k) + \log p(\boldsymbol{\mu}_k|\boldsymbol{\Sigma}_k) 
            + \Big\langle \sum_n^N z_{nk} \log p(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) \Big\rangle_{q(\mathbf{z})}
        \Big] + \text{const}\\

    &= \sum_k^K \Big[ 
            \log p(\boldsymbol{\Sigma}_k) + \log p(\boldsymbol{\mu}_k|\boldsymbol{\Sigma}_k) 
            +  \sum_n^N \Big\langle z_{nk} \Big\rangle_{q(\mathbf{z})} \log p(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) 
        \Big] + \text{const}\\

    &= \sum_k^K \Big[ 
            \log p(\boldsymbol{\Sigma}_k) + \log p(\boldsymbol{\mu}_k|\boldsymbol{\Sigma}_k) 
            +  \sum_n^N r_{nk} \log p(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) 
        \Big] + \text{const}\\   
        
    \implies & \log q(\boldsymbol{\mu},\boldsymbol{\Sigma}) = \sum_k^K q(\mu_k, \Sigma_k)
\end{align}$$

Hence, 

$$\begin{align}
    \log q(\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) &= \underbrace{\log p(\boldsymbol{\Sigma}_k) + \log p(\boldsymbol{\mu}_k|\boldsymbol{\Sigma}_k)}_{\text{NIW}} +  \sum_n^N r_{nk} \log \underbrace{p(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)}_{\text{MVN}} + \text{const}\\

    &= \log \text{NIW}(\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k | \mathbf{m}_{0,k},\kappa_{0,k},\nu_{0,k},\mathbf{S}_{0,k}) + 
       \sum_n^N r_{nk} \log \mathcal{N}(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) + \text{const}\\

    &= \log \Big[ \mathcal{N}(\boldsymbol{\mu}_k|\mathbf{m}_{0,k},\frac{1}{\kappa_{0,k}}\boldsymbol{\Sigma}_k) \cdot \mathcal{IW}(\boldsymbol{\Sigma}_k|\nu_{0,k},\mathbf{S}_{0,k}) \Big] + 
    \sum_n^N r_{nk} \log \mathcal{N}(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) + \text{const}\\\\

    &= \log\underbrace{\big(|\boldsymbol{\Sigma}_k|^{-\frac{\nu_0 + D + 1}{2}}\big) + \Big( -\frac{\kappa_0}{2} (\boldsymbol{\mu}_k - \mathbf{m}_{0,k})^T \boldsymbol{\Sigma}_k^{-1}(\boldsymbol{\mu}_k - \mathbf{m}_{0,k}) -\frac{1}{2} \text{tr}(\boldsymbol{\Sigma}^{-1}\mathbf{S}_{0,k}) \Big)}_{\text{Terms dependent on $\mu_k,\Sigma_k$}}\\
       &\qquad\qquad + \sum_n^N r_{nk} \big(\underbrace{-\frac{1}{2}\log|\boldsymbol{\Sigma}_k| -\frac{1}{2}(\mathbf{x}_n - \boldsymbol{\mu}_k)^T\boldsymbol{\Sigma}_k^{-1}(\mathbf{x}_n - \boldsymbol{\mu}_k)}_{\text{Terms dependent on $\mu_k,\Sigma_k$}}\big) + \text{const}\\\\

    &= \log\big(|\boldsymbol{\Sigma}_k|^{-\frac{\nu_0 + D + 1}{2}}\big) + \Big( -\frac{\kappa_0}{2} (\boldsymbol{\mu}_k - \mathbf{m}_{0,k})^T \boldsymbol{\Sigma}_k^{-1}(\boldsymbol{\mu}_k - \mathbf{m}_{0,k}) -\frac{1}{2} \text{tr}(\boldsymbol{\Sigma}^{-1}\mathbf{S}_{0,k}) \Big)\\
       &\qquad\qquad + \sum_n^N r_{nk} \big(-\frac{1}{2}\log|\boldsymbol{\Sigma}_k|\big) + \sum_n^N r_{nk} \big(-\frac{1}{2}\log|\boldsymbol{\Sigma}_k| -\frac{1}{2}(\mathbf{x}_n - \boldsymbol{\mu}_k)^T\boldsymbol{\Sigma}_k^{-1}(\mathbf{x}_n - \boldsymbol{\mu}_k)\big) + \text{const}\\\\

    &= \log\big(|\boldsymbol{\Sigma}_k|^{-\frac{\nu_0 + D + 1}{2}}\big) + \Big( -\frac{\kappa_0}{2} (\boldsymbol{\mu}_k - \mathbf{m}_{0,k})^T \boldsymbol{\Sigma}_k^{-1}(\boldsymbol{\mu}_k - \mathbf{m}_{0,k}) -\frac{1}{2} \text{tr}(\boldsymbol{\Sigma}^{-1}\mathbf{S}_{0,k}) \Big)\\
       &\qquad\qquad + \sum_n^N  -\frac{r_{nk}}{2}\log|\boldsymbol{\Sigma}_k| + \sum_n^N r_{nk} \big(-\frac{1}{2}\log|\boldsymbol{\Sigma}_k| -\frac{1}{2}(\mathbf{x}_n - \boldsymbol{\mu}_k)^T\boldsymbol{\Sigma}_k^{-1}(\mathbf{x}_n - \boldsymbol{\mu}_k)\big) + \text{const}\\\\
    

    &= \left(-\frac{\nu_0 + D + 1}{2}\right)\log\big|\boldsymbol{\Sigma}_k\big| + \Big( -\frac{\kappa_0}{2} (\boldsymbol{\mu}_k - \mathbf{m}_{0,k})^T \boldsymbol{\Sigma}_k^{-1}(\boldsymbol{\mu}_k - \mathbf{m}_{0,k}) -\frac{1}{2} \text{tr}(\boldsymbol{\Sigma}^{-1}\mathbf{S}_{0,k}) \Big)\\
       &\qquad\qquad + \log|\boldsymbol{\Sigma}_k | \sum_n^N  -\frac{r_{nk}}{2} + \sum_n^N r_{nk} \big(-\frac{1}{2}(\mathbf{x}_n - \boldsymbol{\mu}_k)^T\boldsymbol{\Sigma}_k^{-1}(\mathbf{x}_n - \boldsymbol{\mu}_k)\big) + \text{const}\\\\

    &= \left(-\frac{\nu_0 + D + 1}{2}\right)\log\big|\boldsymbol{\Sigma}_k\big| +
        \Big( -\frac{\kappa_0}{2} (\boldsymbol{\mu}_k - \mathbf{m}_{0,k})^T \boldsymbol{\Sigma}_k^{-1}(\boldsymbol{\mu}_k - \mathbf{m}_{0,k}) -\frac{1}{2} \text{tr}(\boldsymbol{\Sigma}^{-1}\mathbf{S}_{0,k}) \Big)\\
            &\qquad\qquad \log|\boldsymbol{\Sigma}_k|^{\frac{-N_k}{2}}- \frac{1}{2}\Big[\text{tr}(\boldsymbol{\Sigma}_k^{-1}\mathbf{S}_{\bar{\mathbf{x}}_k}) + N_k(\bar{\mathbf{x}}_k - \boldsymbol{\mu}_k)^T\boldsymbol{\Sigma}_k^{-1}(\bar{\mathbf{x}}_k - \boldsymbol{\mu}_k)\Big] + \text{const}, \\
            &\qquad\qquad \text{where } N_k = \sum_n^N r_{nk}, \quad  \bar{\mathbf{x}}_k = \frac{1}{N_k} \sum_n^N r_{nk}\cdot\mathbf{x}_n,\quad \mathbf{S}_{\bar{\mathbf{x}}_k} = \sum_n^N r_{nk}(\mathbf{x}_n - \bar{\mathbf{x}}_k)(\mathbf{x}_n - \bar{\mathbf{x}}_k)^T\\\\

    &= \left(-\frac{\nu_0 + N_k + D + 1}{2}\right)\log\big|\boldsymbol{\Sigma}_k\big| + 
       \Big[  -\frac{\kappa_0}{2} \big( \boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k - 2\boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\mathbf{m}_{0,k} + \mathbf{m}_{0,k}^T\boldsymbol{\Sigma}_k^{-1}\mathbf{m}_{0,k} \big) -\frac{1}{2} \text{tr}(\boldsymbol{\Sigma}^{-1}\mathbf{S}_{0,k}) \Big]\\

        &\qquad - \frac{1}{2}\Big[\text{tr}(\boldsymbol{\Sigma}_k^{-1}\mathbf{S}_{\bar{\mathbf{x}}_k}) + N_k\big(\bar{\mathbf{x}}_k^T\boldsymbol{\Sigma}_k^{-1}\bar{\mathbf{x}}_k - 2\bar{\mathbf{x}}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k + \boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k\big)\Big] + \text{const}\\\\

    &= -\frac{1}{2}\Bigg[
            (\nu_0 + N_k + D + 1)\log\big|\boldsymbol{\Sigma}_k\big| + 
              \kappa_0 \big( \boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k - 2\boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\mathbf{m}_{0,k} + \mathbf{m}_{0,k}^T\boldsymbol{\Sigma}_k^{-1}\mathbf{m}_{0,k} \big) + \text{tr}(\boldsymbol{\Sigma}^{-1}\mathbf{S}_{0,k}) \\

            &\qquad + \text{tr}(\boldsymbol{\Sigma}_k^{-1}\mathbf{S}_{\bar{\mathbf{x}}_k}) + N_k\big(\bar{\mathbf{x}}_k^T\boldsymbol{\Sigma}_k^{-1}\bar{\mathbf{x}}_k - 2\bar{\mathbf{x}}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k + \boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k\big)
        \Bigg] + \text{const}\\\\

    &= -\frac{1}{2}\Bigg[
            (\nu_0 + N_k + D + 1)\log\big|\boldsymbol{\Sigma}_k\big| +
                \text{tr}(\boldsymbol{\Sigma}_k^{-1}\mathbf{S}_{0,k} + \boldsymbol{\Sigma}_k^{-1}\mathbf{S}_{\bar{x}_k}) + \underbrace{\kappa_0\boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k}_{(1)} - \underbrace{2\kappa_0\boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\mathbf{m}_{0,k}}_{(2)} + \underbrace{\kappa_0\mathbf{m}_{0,k}^T\boldsymbol{\Sigma}_k^{-1}\mathbf{m}_{0,k}}_{(3)} + \\

                &\qquad\qquad \underbrace{N_k\bar{\mathbf{x}}_k^T\boldsymbol{\Sigma}_k^{-1}\bar{\mathbf{x}}_k}_{(3)} - \underbrace{2N_k\bar{\mathbf{x}}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k}_{(2)} + \underbrace{N_k\boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k}_{(1)}
        \Bigg] + \text{const}, \qquad\qquad \text{Identify like terms}\\\\
    
    &= -\frac{1}{2}\Bigg[
            (\nu_0 + N_k + D + 1)\log\big|\boldsymbol{\Sigma}_k\big| + \text{tr}\Big(\boldsymbol{\Sigma}_k^{-1}(\mathbf{S}_{0,k} + \mathbf{S}_{\bar{x}_k})\Big) + \\
            
                &\qquad\qquad (\kappa_0 + N_k)\boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k - 2\Big( \kappa_0\boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\mathbf{m}_{0,k} + N_k\bar{\mathbf{x}}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k \Big) + \kappa_0\mathbf{m}_{0,k}^T\boldsymbol{\Sigma}_k^{-1}\mathbf{m}_{0,k} + N_k\bar{\mathbf{x}}_k^T\boldsymbol{\Sigma}_k^{-1}\bar{\mathbf{x}}_k
        \Bigg] + \text{const}\\\\

    &= -\frac{1}{2}\Bigg[
            (\nu_0 + N_k + D + 1)\log\big|\boldsymbol{\Sigma}_k\big| + \text{tr}\Big(\boldsymbol{\Sigma}_k^{-1}(\mathbf{S}_{0,k} + \mathbf{S}_{\bar{x}_k})\Big) + \\

            &\qquad\qquad \boldsymbol{\mu}_k^T\underbrace{\Big((\kappa_0 + N_k)\boldsymbol{\Sigma}_k^{-1}\Big)}_{\mathbf{M}}\boldsymbol{\mu}_k - 2\boldsymbol{\mu}_k^T\underbrace{\Big( \kappa_0\boldsymbol{\Sigma}_k^{-1}\mathbf{m}_{0,k} + N_k\boldsymbol{\Sigma}_k^{-1}\mathbf{\bar{x}}_k  \Big)}_{\mathbf{b}} + \kappa_0\mathbf{m}_{0,k}^T\boldsymbol{\Sigma}_k^{-1}\mathbf{m}_{0,k} + N_k\bar{\mathbf{x}}_k^T\boldsymbol{\Sigma}_k^{-1}\bar{\mathbf{x}}_k
        \Bigg] + \text{const}\\\\

    &= -\frac{1}{2}\Bigg[
            (\nu_0 + N_k + D + 1)\log\big|\boldsymbol{\Sigma}_k\big| + \text{tr}\Big(\boldsymbol{\Sigma}_k^{-1}(\mathbf{S}_{0,k} + \mathbf{S}_{\bar{x}_k})\Big) + \\

            &\qquad\qquad \underline{(\boldsymbol{\mu}_k - \mathbf{M}^{-1}\mathbf{b})^T\mathbf{M}(\boldsymbol{\mu}_k - \mathbf{M}^{-1}\mathbf{b})} + \kappa_0\mathbf{m}_{0,k}^T\boldsymbol{\Sigma}_k^{-1}\mathbf{m}_{0,k} + N_k\bar{\mathbf{x}}_k^T\boldsymbol{\Sigma}_k^{-1}\bar{\mathbf{x}}_k
        \Bigg] + \text{const},\qquad \text{Complete the square}\\
        &\text{where,}\\
            &\qquad\qquad \mathbf{M} = (\kappa_0 + N_k)\boldsymbol{\Sigma}_k^{-1}\\
            &\qquad\qquad \mathbf{b} = (\kappa_0\boldsymbol{\Sigma}_k^{-1}\mathbf{m}_{0,k} + N_k\boldsymbol{\Sigma}_k^{-1}\mathbf{\bar{x}}_k)\\\\

    &= -\frac{1}{2}\Bigg[
            (\nu_0 + N_k + D + 1)\log\big|\boldsymbol{\Sigma}_k\big| + \text{tr}\Big(\boldsymbol{\Sigma}_k^{-1}(\mathbf{S}_{0,k} + \mathbf{S}_{\bar{x}_k})\Big) + \\

            &\qquad\qquad (\boldsymbol{\mu}_k - \mathbf{M}^{-1}\mathbf{b})^T\mathbf{M}(\boldsymbol{\mu}_k - \mathbf{M}^{-1}\mathbf{b}) + \text{tr}\Big(\kappa_0\mathbf{m}_{0,k}^T\boldsymbol{\Sigma}_k^{-1}\mathbf{m}_{0,k}\Big) + \text{tr}\Big(N_k\bar{\mathbf{x}}_k^T\boldsymbol{\Sigma}_k^{-1}\bar{\mathbf{x}}_k\Big)
        \Bigg] + \text{const}\\\\

    &= -\frac{1}{2}\Bigg[
            (\nu_0 + N_k + D + 1)\log\big|\boldsymbol{\Sigma}_k\big| + \text{tr}\Big(\boldsymbol{\Sigma}_k^{-1}(\mathbf{S}_{0,k} + \mathbf{S}_{\bar{x}_k})\Big) + \\

            &\qquad\qquad (\boldsymbol{\mu}_k - \mathbf{M}^{-1}\mathbf{b})^T\mathbf{M}(\boldsymbol{\mu}_k - \mathbf{M}^{-1}\mathbf{b}) + \text{tr}\Big(\kappa_0\boldsymbol{\Sigma}_k^{-1}\mathbf{m}_{0,k}\mathbf{m}_{0,k}^T\Big) + \text{tr}\Big(N_k\boldsymbol{\Sigma}_k^{-1}\bar{\mathbf{x}}_k\bar{\mathbf{x}}_k^T\Big)
        \Bigg] + \text{const}, \qquad \text{Trace cyclic property}\\\\

    &= -\frac{1}{2}\Bigg[
            (\nu_0 + N_k + D + 1)\log\big|\boldsymbol{\Sigma}_k\big| + \text{tr}\Big(\boldsymbol{\Sigma}_k^{-1}(\mathbf{S}_{0,k} + \mathbf{S}_{\bar{x}_k} + \kappa_0\mathbf{m}_{0,k}\mathbf{m}_{0,k}^T + N_k\bar{\mathbf{x}}_k\bar{\mathbf{x}}_k^T ) \Big) + \\

            &\qquad\qquad (\boldsymbol{\mu}_k - \mathbf{M}^{-1}\mathbf{b})^T\mathbf{M}(\boldsymbol{\mu}_k - \mathbf{M}^{-1}\mathbf{b})
        \Bigg] + \text{const}\\\\

    &= -\frac{1}{2}\Bigg[
            (\nu_0 + N_k + D + 1)\log\big|\boldsymbol{\Sigma}_k\big| + \text{tr}\Big(\boldsymbol{\Sigma}_k^{-1}(\mathbf{S}_{0,k} + \mathbf{S}_{\bar{x}_k} + \kappa_0\mathbf{m}_{0,k}\mathbf{m}_{0,k}^T + N_k\bar{\mathbf{x}}_k\bar{\mathbf{x}}_k^T ) \Big) + \\

            &\qquad\qquad (\boldsymbol{\mu}_k - \mathbf{M}^{-1}\mathbf{b})^T\Big(\underline{(\kappa_0 + N_k)\boldsymbol{\Sigma}^{-1}}\Big)(\boldsymbol{\mu}_k - \mathbf{M}^{-1}\mathbf{b})
        \Bigg] + \text{const}\\\\

    \implies & q(\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) = \big|\boldsymbol{\Sigma}_k\big|^{-\frac{\nu_0 + N_k + D + 1}{2}} 
                \exp \Big( -\frac{\kappa_0 + N_k}{2}(\boldsymbol{\mu}_k - \mathbf{M}^{-1}\mathbf{b})^T\boldsymbol{\Sigma}^{-1}(\boldsymbol{\mu}_k - \mathbf{M}^{-1}\mathbf{b})  -\frac{1}{2}\text{tr}\Big(\boldsymbol{\Sigma}_k^{-1}\mathbf{W}_k \Big) \Big) \times \underbrace{\text{const}}_{\text{Normalization}}\\

            &\text{where},\\
                &\qquad\qquad \mathbf{W}_k = \mathbf{S}_{0,k} + \mathbf{S}_{\bar{x}_k} + \kappa_0\mathbf{m}_{0,k}\mathbf{m}_{0,k}^T + N_k\bar{\mathbf{x}}_k\bar{\mathbf{x}}_k^T 
\end{align}$$

This is Normal Inverse Wishart form. Hence,

$$\begin{align}
   & q(\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) = \mathcal{N}(\boldsymbol{\mu}_k|\mathbf{m}_{k},\frac{1}{\kappa_k}\boldsymbol{\Sigma}_k)\mathcal{IW}(\boldsymbol{\Sigma}_k|\mathbf{W}_k,\nu_k)\\

    &\qquad\qquad \mathbf{m}_k = \mathbf{M}^{-1}\mathbf{b} = \frac{1}{\kappa_k}(\kappa_0\mathbf{m}_{0,k} + N_k\mathbf{\bar{x}}_k)\\
    &\qquad\qquad \kappa_k = \kappa_0 + N_k\\
    &\qquad\qquad \mathbf{W}_k =  \mathbf{S}_{0,k} + \mathbf{S}_{\bar{x}_k} + \kappa_0\mathbf{m}_{0,k}\mathbf{m}_{0,k}^T + N_k\bar{\mathbf{x}}_k\bar{\mathbf{x}}_k^T \\
    &\qquad\qquad \nu_k = \nu_0 + N_k
\end{align}$$

#### Update step for $q(\mathbf{z})$

$  p(\mathbf{X}|\boldsymbol{\mu},\boldsymbol{\Sigma},\mathbf{z})p(\boldsymbol{\mu}|\boldsymbol{\Sigma})p(\boldsymbol{\Sigma})p(\mathbf{z}|\boldsymbol{\pi})p(\boldsymbol{\pi})$

$$\begin{align}
    \log q(\mathbf{z}) &= \Big\langle \log p(\mathbf{X},\boldsymbol{\mu},\boldsymbol{\Sigma},\boldsymbol{\pi},\mathbf{z}) \Big\rangle_{q(\mu,\Sigma,\pi)} + \text{const}\\\\

    &= \Big\langle \log p(\mathbf{X}|\boldsymbol{\mu},\boldsymbol{\Sigma},\mathbf{z})p(\boldsymbol{\mu}|\boldsymbol{\Sigma})p(\boldsymbol{\Sigma}) \Big\rangle_{q(\mu,\Sigma)} + 
       \Big\langle \log p(\mathbf{z}|\boldsymbol{\pi})p(\boldsymbol{\pi}) \Big\rangle_{q(\pi)} + 
       \text{const}\\\\

    &= \Big\langle \log p(\mathbf{X}|\boldsymbol{\mu},\boldsymbol{\Sigma},\mathbf{z}) \Big\rangle_{q(\mu,\Sigma)} + 
       \Big\langle \log p(\mathbf{z}|\boldsymbol{\pi}) \Big\rangle_{q(\pi)} + 
       \underbrace{\Big\langle \log p(\boldsymbol{\mu}|\boldsymbol{\Sigma})p(\boldsymbol{\Sigma}) \Big\rangle_{q(\mu,\Sigma)} + 
       \Big\langle \log p(\boldsymbol{\pi}) \Big\rangle_{q(\pi)}}_{\text{Terms independent of } \mathbf{z}} +
       \text{const}\\\\

    &=  \Big\langle \log p(\mathbf{X}|\boldsymbol{\mu},\boldsymbol{\Sigma},\mathbf{z}) \Big\rangle_{q(\mu,\Sigma)} + 
        \Big\langle \log p(\mathbf{z}|\boldsymbol{\pi}) \Big\rangle_{q(\pi)} + \text{const},\qquad \text{Independent terms absorbed into constant}\\\\

    &=  \Big\langle \sum_n^N \sum_k^K \log p(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k)^{z_{nk}} \Big\rangle_{q(\mu,\Sigma)} + 
        \Big\langle \sum_n^N \sum_k^K \log p(z_n=k|\boldsymbol{\pi})^{z_{nk}} \Big\rangle_{q(\pi)} + 
        \text{const}\\\\

    &=  \Big\langle \sum_n^N \sum_k^K z_{nk} \cdot \log p(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) \Big\rangle_{q(\mu,\Sigma)} + 
        \Big\langle \sum_n^N \sum_k^K z_{nk} \cdot \log \boldsymbol{\pi}_k \Big\rangle_{q(\pi)} + 
        \text{const}\\\\

    &= \sum_n^N \Bigg[ 
            \sum_k^K \Big\langle z_{nk} \cdot \log p(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) \big\rangle_{q(\mu,\Sigma)} +
            \sum_k^K \Big\langle z_{nk} \cdot \log \boldsymbol{\pi}_k  \Big\rangle_{q(\pi)}
        \Bigg] + \text{const}\\\\

    \implies & \log q(\mathbf{z}) = \sum_n^N \log q(z_n)

\end{align}$$

Hence,

$$\begin{align}
    \log q(z_n) &= \sum_k^K \Big\langle z_{nk} \cdot \log p(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) \big\rangle_{q(\mu,\Sigma)} +
                   \sum_k^K \Big\langle z_{nk} \cdot \log \boldsymbol{\pi}_k  \Big\rangle_{q(\pi)} + \text{const}\\\\

        &=  \sum_k^K \Big[ 
            \Big\langle z_{nk} \cdot \log p(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) \big\rangle_{q(\mu,\Sigma)} + 
            \Big\langle z_{nk} \cdot \log \boldsymbol{\pi}_k  \Big\rangle_{q(\pi)}
        \Big] + \text{const}\\\\
\end{align}$$

This gives
$$\begin{align}
     \log q(z_n=k) &= \underbrace{\Big\langle \log p(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) \big\rangle_{q(\mu,\Sigma)}}_{(1)} + 
                      \underbrace{\Big\langle \log \boldsymbol{\pi}_k  \Big\rangle_{q(\pi)}}_{(2)} + \text{const}\\\\

\end{align}$$

where

$$\begin{align}
    (1)\quad \Big\langle \log p(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) \big\rangle_{q(\mu,\Sigma)} &=
                \Big\langle \log \mathcal{N}(\mathbf{x}_n|\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) \Big\rangle_{q(\mu,\Sigma)}\\\\
            
        &= \Big\langle \underbrace{-\frac{1}{2}\log|\boldsymbol{\Sigma}_k| -\frac{1}{2}(\mathbf{x}_n - \boldsymbol{\mu}_k)^T\boldsymbol{\Sigma}_k^{-1}(\mathbf{x}_n - \boldsymbol{\mu}_k)}_{\text{Terms dependent on $\mu_k,\Sigma_k$}} \Big\rangle_{q(\mu,\Sigma)},
        \quad \text{Independent terms absorbed to const}\\\\

        &= -\frac{1}{2} \Big\langle \log|\boldsymbol{\Sigma}_k| + (\mathbf{x}_n - \boldsymbol{\mu}_k)^T\boldsymbol{\Sigma}_k^{-1}(\mathbf{x}_n - \boldsymbol{\mu}_k) \Big\rangle_{q(\mu,\Sigma)}\\\\

        &= -\frac{1}{2} \Big\langle \log|\boldsymbol{\Sigma}_k| + \mathbf{x}_n^T\boldsymbol{\Sigma}_k^{-1}\mathbf{x}_n - 2\mathbf{x}_n^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k + \boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k \Big\rangle_{q(\mu,\Sigma)}\\\\

        &= -\frac{1}{2} \Bigg[ 
                \Big\langle \log|\boldsymbol{\Sigma}_k| \Big\rangle_{q(\Sigma)} + 
                \Big\langle \mathbf{x}_n^T\boldsymbol{\Sigma}_k^{-1}\mathbf{x}_n \Big\rangle_{q(\Sigma)} - 
                \Big\langle 2\mathbf{x}_n^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k \Big\rangle_{q(\mu,\Sigma)} + 
                \Big\langle \boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k \Big\rangle_{q(\mu,\Sigma)}
            \Bigg]\\\\

        &= -\frac{1}{2} \Bigg[
                \big\langle \log|\boldsymbol{\Sigma}_k| \big\rangle_{q(\Sigma)} +
                \mathbf{x}_n^T \big\langle \boldsymbol{\Sigma}_k^{-1} \big\rangle_{q(\Sigma)} \mathbf{x}_n  - 
                2\mathbf{x}_n^T\big\langle \boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k \big\rangle_{q(\mu,\Sigma)} + 
                \Big\langle \boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k \Big\rangle_{q(\mu,\Sigma)}
           \Bigg]\\\\

        \text{Given that } \boldsymbol{\Sigma}_k &\sim \mathcal{IW}(\boldsymbol{\Sigma_k} | \mathbf{W}_k, \nu_k), \langle \boldsymbol{\Sigma}_k^{-1} \rangle = \nu_k \mathbf{W}_k^{-1}, \text{ and } \boldsymbol{\mu}_k \sim \mathcal{N}(\boldsymbol{\mu}_k | \mathbf{m}_k, \frac{1}{\kappa_k}\boldsymbol{\Sigma}_k)\\\\

        &= -\frac{1}{2} \Bigg[
            \big\langle \log|\boldsymbol{\Sigma}_k| \big\rangle_{q(\Sigma)} +
            \mathbf{x}_n^T (\nu_k \mathbf{W}_k^{-1}) \mathbf{x}_n  - 
            2\mathbf{x}_n^T(\nu_k \mathbf{W}_k^{-1}) \mathbf{m}_k + 
            \Big\langle \boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k \Big\rangle_{q(\mu,\Sigma)}
        \Bigg]\\\\

        &= -\frac{1}{2} \Bigg[
            \big\langle \log|\boldsymbol{\Sigma}_k| \big\rangle_{q(\Sigma)} +
            \mathbf{x}_n^T (\nu_k \mathbf{W}_k^{-1}) \mathbf{x}_n  - 
            2\mathbf{x}_n^T(\nu_k \mathbf{W}_k^{-1}) \mathbf{m}_k + 
            \Big\langle \text{tr}\big( \boldsymbol{\mu}_k^T\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k \big)\Big\rangle_{q(\mu,\Sigma)}
        \Bigg]\\\\

        &= -\frac{1}{2} \Bigg[
            \cdots + 
            \Big\langle \text{tr}\big( \boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k\boldsymbol{\mu}_k^T \big)\Big\rangle_{q(\mu,\Sigma)}
        \Bigg],\quad \text{Trace cyclic property}\\\\

        &= -\frac{1}{2} \Bigg[
            \cdots + 
            \text{tr}\Big( \big\langle  \boldsymbol{\Sigma}_k^{-1}\boldsymbol{\mu}_k\boldsymbol{\mu}_k^T \big\rangle_{q(\mu,\Sigma)} \Big)
        \Bigg] \\\\
        
        &= -\frac{1}{2} \Bigg[
            \cdots + 
            \text{tr}\Big( \Big\langle  \boldsymbol{\Sigma}_k^{-1} \big\langle\boldsymbol{\mu}_k\boldsymbol{\mu}_k^T \big\rangle_{q(\mu|\Sigma)} \Big\rangle_{q(\Sigma)}  \Big)
        \Bigg] \\\\

        &= -\frac{1}{2} \Bigg[
            \cdots + 
            \text{tr}\Big( \Big\langle  \boldsymbol{\Sigma}_k^{-1} (\mathbf{m}_k\mathbf{m}_k^T + \frac{1}{\kappa_k}\boldsymbol{\Sigma}_k) \Big\rangle_{q(\Sigma)}  \Big)
        \Bigg]\\\\
        
        &= -\frac{1}{2} \Bigg[
            \cdots + 
            \text{tr}\Big( \Big\langle   \boldsymbol{\Sigma}_k^{-1}\mathbf{m}_k\mathbf{m}_k^T + \frac{1}{\kappa_k}\boldsymbol{\Sigma}_k^{-1}\boldsymbol{\Sigma}_k \Big\rangle_{q(\Sigma)}  \Big)
        \Bigg]  \\\\

        &= -\frac{1}{2} \Bigg[
            \cdots + 
            \text{tr}\Big( \Big\langle \boldsymbol{\Sigma}_k^{-1}\Big\rangle_{q(\Sigma)}\mathbf{m}_k\mathbf{m}_k^T + \frac{1}{\kappa_k}I   \Big)
        \Bigg]\\\\

        &= -\frac{1}{2} \Bigg[
            \cdots + 
            \text{tr}\Big( (\nu_k \mathbf{W}_k^{-1})\mathbf{m}_k\mathbf{m}_k^T + \frac{1}{\kappa_k}I \Big)
        \Bigg] \\\\
        
        &= -\frac{1}{2} \Bigg[
            \cdots + 
            \text{tr}\Big( \nu_k\mathbf{W}_k^{-1}\mathbf{m}_k\mathbf{m}_k^T \Big) + 
            \text{tr}\Big(\frac{1}{\kappa_k}I \Big)
        \Bigg] \\\\


        &= -\frac{1}{2} \Bigg[
            \cdots + 
            \underbrace{\text{tr}\Big( \nu_k \mathbf{m}_k^T \mathbf{W}_k^{-1}\mathbf{m}_k \Big)}_{\text{Scalar}} + 
            \frac{D}{\kappa_k}
        \Bigg],\qquad \text{Trace cyclic property} \\\\

        &= -\frac{1}{2} \Bigg[
            \cdots + 
            \nu_k \mathbf{m}_k^T \mathbf{W}_k^{-1}\mathbf{m}_k  +
            \frac{D}{\kappa_k}
        \Bigg] \\\\

        &= -\frac{1}{2} \Bigg[
            \big\langle \log|\boldsymbol{\Sigma}_k| \big\rangle_{q(\Sigma)} +
            \mathbf{x}_n^T (\nu_k \mathbf{W}_k^{-1}) \mathbf{x}_n  - 
            2\mathbf{x}_n^T(\nu_k \mathbf{W}_k^{-1}) \mathbf{m}_k + 
           \nu_k \mathbf{m}_k^T \mathbf{W}_k^{-1}\mathbf{m}_k  + 
           \frac{D}{\kappa_k}
        \Bigg] \\\\


        &=  -\frac{1}{2} \Bigg[
             \log|\mathbf{W}_k| - \sum_i^D \psi \left( \frac{\nu_k + 1 - i}{2} \right) - D\log2 + \\

            &\qquad\qquad \mathbf{x}_n^T (\nu_k \mathbf{W}_k^{-1}) \mathbf{x}_n  - 
            2\mathbf{x}_n^T(\nu_k \mathbf{W}_k^{-1}) \mathbf{m}_k + 
            \nu_k \mathbf{m}_k^T \mathbf{W}_k^{-1}\mathbf{m}_k  + 
            \frac{D}{\kappa_k}

        \Bigg] \\\\

        % &= -\frac{1}{2} \log|\mathbf{W}_k| +  \frac{1}{2}\sum_i^D \psi \left( \frac{\nu_k + 1 - i}{2} \right) + \frac{D}{2}\log2 - \\
        %     &\qquad\qquad \frac{1}{2}\mathbf{x}_n^T (\nu_k \mathbf{W}_k^{-1}) \mathbf{x}_n + 
        %     \mathbf{x}_n^T(\nu_k \mathbf{W}_k^{-1}) \mathbf{m}_k - 
        %     \frac{1}{2}\nu_k \mathbf{m}_k^T \mathbf{W}_k^{-1}\mathbf{m}_k - 
        %     \frac{D}{2\kappa_k}\\\\

\end{align}$$

For (2), given that each marginal of a Dirichlet distribution is a Beta distribution with : $\boldsymbol{\pi}_k \sim \text{Beta}(\alpha_k, \sum_{k'}\alpha_{k'}-\alpha_k)$  
And that if $X\sim\text{Beta}(a,b)$, then $\mathbb{E}[\log X] = \psi(a) - \psi(a + b)$
$$\begin{align}
    (2) \quad \mathbb{E}_{q(\boldsymbol{\pi})}\Big[ \log \boldsymbol{\pi}_{k} \Big] = \psi(\alpha_k) - \psi\Big(\sum_{k'}\alpha_{k'}\Big)
\end{align}$$

Hence we get the update step : 
$$\begin{align}
    \log q(z_n=k) &= -\frac{1}{2} \Bigg[
                                        \log|\mathbf{W}_k| - \sum_i^D \psi \left( \frac{\nu_k + 1 - i}{2} \right) - D\log2 + 
                                        \mathbf{x}_n^T (\nu_k \mathbf{W}_k^{-1}) \mathbf{x}_n  - 
                                        2\mathbf{x}_n^T(\nu_k \mathbf{W}_k^{-1}) \mathbf{m}_k + 
                                        \nu_k \mathbf{m}_k^T \mathbf{W}_k^{-1}\mathbf{m}_k  + 
                                        \frac{D}{\kappa_k}

                                    \Bigg] \\
                                    
                        &\qquad\qquad + \psi(\alpha_k) - \psi\Big(\sum_{k'}\alpha_{k'}\Big) + \text{const}\\\\

    \implies & q(z_n=k) =  \exp \Bigg( \psi(\alpha_k) - \psi\big(\sum_{k'}\alpha_{k'}\big)\\
                                                    &\qquad\qquad -\frac{1}{2} \Bigg[
                                                    \log|\mathbf{W}_k| - \sum_i^D \psi \left( \frac{\nu_k + 1 - i}{2} \right) - D\log2 + 
                                                    \mathbf{x}_n^T (\nu_k \mathbf{W}_k^{-1}) \mathbf{x}_n  - 
                                                    2\mathbf{x}_n^T(\nu_k \mathbf{W}_k^{-1}) \mathbf{m}_k + 
                                                    \nu_k \mathbf{m}_k^T \mathbf{W}_k^{-1}\mathbf{m}_k  + 
                                                    \frac{D}{\kappa_k}

                                                \Bigg]  + \underbrace{\text{const}}_{\text{Normalization}}
                                \Bigg)
\end{align}$$

### Summary

$$\begin{align}
    &\boxed{q(\boldsymbol{\pi}) = \prod_k^K \pi_k^{\alpha_0 + N_k - 1} + \text{const} = \text{Dir}(\boldsymbol{\pi}|\boldsymbol{\alpha}_0 + N), \quad \text{where } N = [N_0,...,N_K]}\\\\

    & \boxed{q(\boldsymbol{\mu}_k,\boldsymbol{\Sigma}_k) = \mathcal{N}(\boldsymbol{\mu}_k|\mathbf{m}_{k},\frac{1}{\kappa_k}\boldsymbol{\Sigma}_k)\mathcal{IW}(\boldsymbol{\Sigma}_k|\mathbf{W}_k,\nu_k)}\\

    &\qquad\qquad \mathbf{m}_k = \mathbf{M}^{-1}\mathbf{b} = \frac{1}{\kappa_k}(\kappa_0\mathbf{m}_{0,k} + N_k\mathbf{\bar{x}}_k)\\
    &\qquad\qquad \kappa_k = \kappa_0 + N_k\\
    &\qquad\qquad \mathbf{W}_k =  \mathbf{S}_{0,k} + \mathbf{S}_{\bar{x}_k} + \kappa_0\mathbf{m}_{0,k}\mathbf{m}_{0,k}^T + N_k\bar{\mathbf{x}}_k\bar{\mathbf{x}}_k^T \\
    &\qquad\qquad \nu_k = \nu_0 + N_k\\\\

    & \boxed{\frac{q(z_n=k) =  \exp \Bigg( \psi(\alpha_k) - \psi\big(\sum_{k'}\alpha_{k'}\big) -\frac{1}{2} \Bigg[
                                        \log|\mathbf{W}_k| - \sum_i^D \psi \left( \frac{\nu_k + 1 - i}{2} \right) - D\log2 + 
                                        \mathbf{x}_n^T (\nu_k \mathbf{W}_k^{-1}) \mathbf{x}_n  - 
                                        2\mathbf{x}_n^T(\nu_k \mathbf{W}_k^{-1}) \mathbf{m}_k + 
                                        \nu_k \mathbf{m}_k^T \mathbf{W}_k^{-1}\mathbf{m}_k  + 
                                        \frac{D}{\kappa_k}

                                    \Bigg]
                                \Bigg)}{\sum_k^K
                                        \exp \Bigg( \psi(\alpha_k) - \psi\big(\sum_{k'}\alpha_{k'}\big) -\frac{1}{2} \Bigg[
                                        \log|\mathbf{W}_k| - \sum_i^D \psi \left( \frac{\nu_k + 1 - i}{2} \right) - D\log2 + 
                                        \mathbf{x}_n^T (\nu_k \mathbf{W}_k^{-1}) \mathbf{x}_n  - 
                                        2\mathbf{x}_n^T(\nu_k \mathbf{W}_k^{-1}) \mathbf{m}_k + 
                                        \nu_k \mathbf{m}_k^T \mathbf{W}_k^{-1}\mathbf{m}_k  + 
                                        \frac{D}{\kappa_k}

                                        \Bigg]
                                \Bigg)
                                }}
\end{align}$$